In [16]:
#IMPORTS
from dataset import Dataset_maker
from traindataset import TrainDataset
from torch.utils.data import DataLoader
from model_detector import Detector
import torch

In [2]:
#DATA MAKER
path_wider_images = 'WIDER/WIDER_train/WIDER_train/images/'
path_annot = 'WIDER/wider_face_split/wider_face_split/wider_face_train_bbx_gt.txt'
dm = Dataset_maker(path_wider_images, path_annot)

In [3]:
#CREATE FOLDERS
number = 5 #number of images from each folder
path_for_save = 'C:/Users/Artem/Desktop/Artem/NSU/BMM_project/Faces_for_train_faster_rcnn'
dm.create_folders(path_for_save, number)

305 images saved successfully


C:\Users\Artem\anaconda3\lib\site-packages\numpy\lib\npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [4]:
#CREATE AND SAVE DATAFRAME FOR TRAIN DATASET
dm.dataframe('faces_for_train.csv')

DataFrame saved successfully


In [5]:
#SAVED DATAFRAME
dm.get_dataframe()

,id,xmin,ymin,xmax,ymax
0,0,433,189,467,231
1,0,88,203,141,267
2,0,5,203,36,236
3,0,296,174,341,226
4,0,219,166,254,212
...,...,...,...,...,...
2613,304,848,263,874,295
2614,304,892,313,924,365
2615,304,641,300,658,340
2616,304,554,321,590,375


In [8]:
#TRAIN DATASET FOR DATALOADER
train_dataset = TrainDataset('faces_for_train.csv', path_for_save)

In [11]:
#DATALOADER
def collate_fn(batch):
    return tuple(zip(*batch))

train_data_loader = DataLoader(
    train_dataset,
    batch_size=2,
    shuffle=True,
    collate_fn=collate_fn
)

In [14]:
#MODEL Faster R-CNN
detector = Detector()

In [18]:
#DEVICE
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [17]:
#OPTIMIZER
params = [p for p in detector.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
num_epochs = 10

In [20]:
#TRAINING MODEL
itr = 1
print(f'Dataset length: {train_dataset.__len__()}')
for epoch in range(num_epochs):
    
    for images, targets in train_data_loader:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        loss_dict = detector(images, targets)
        
        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()
        
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        
        if itr % 50 == 0:
            print(f'Iteration #{itr} loss: {loss_value}')
            
        itr+=1
        
    print(f'Epoch #{epoch} loss: {loss_value}')
    
    torch.save(model.state_dict(), f'detector{epoch}.pth') #SAVE WEIGHTS EACH EPOCH

C:\Users\Artem\anaconda3\lib\site-packages\torch\functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Iteration #1 loss: 1.221899151802063


KeyboardInterrupt: 